# TensorFlow and other tools for ML in Julia

**Lyndon White**
 - Research Software Engineer -- Invenia Labs, Cambridge
 - Technically still PhD Candidate -- The University of Western Australia


In [1]:
using Pkg: @pkg_str
pkg"activate  ."

In [2]:
using TensorFlow
using TensorFlow: summary

┌ Info: Recompiling stale cache file /Users/oxinabox/.julia/compiled/v1.1/TensorFlow/IhIhf.ji for TensorFlow [1d978283-2c37-5f34-9a8e-e9c0ece82495]
└ @ Base loading.jl:1184
┌ Warning: Loading a new version of TensorFlow.jl for the first time. This initial load can take around 5 minutes as code is precompiled; subsequent usage will only take a few seconds.
└ @ TensorFlow ~/Documents/oxinabox.github.io/_drafts/JuliaDeepLearningMeetupLondon2019/dev/TensorFlow/src/TensorFlow.jl:3
┌ Warning: Package TensorFlow does not have Random in its dependencies:
│ - If you have TensorFlow checked out for development and have
│   added Random as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with TensorFlow
└ Loading Random into TensorFlow from project dependency, future warnings for TensorFlow are suppressed.


## 4 Types of Nodes, i.e. `Tensors`
 - **Placeholders:** this is where you put your inputs
 - **Operations:** theres transform inputs into outputs, they do math
 - **Variables:** thes arre the things you train, they are mutable
 - **Actions:** These are operations with side effects, like logging (TensorBoard)) and mutating Variable (Optimizers)

## Functions

Functions mutate **the graph** to introduce nodes.

For example:
 - `sin(::Float64)` in julia would return a `Float64` that is the answer.
 - `sin(::Tensor)` introduces a `sin` operation into the graph, and returns a `Tensor` that is a reference to it's output, this could be feed to other operations.
 
The answer to that operation is not computed, until you execute the graph.


In [3]:
sess= Session(Graph())

@tf begin
    x = placeholder(Float64)
    y = sin(x)
end

@show y

run(sess, y, Dict(x=>0.5))

y = <Tensor y:1 shape=unknown dtype=Float64>


2019-01-30 23:25:19.806525: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.2 AVX AVX2 FMA


0.479425538604203

In [19]:
# Create a summary writer
sess= Session(Graph())

@tf begin
    x = placeholder(Float64)
    y = sin(x)
end

@show y

run(sess, y, Dict(x=>0.5))
summary_writer = TensorFlow.summary.FileWriter(mkpath("logs"); graph=sess.graph)
x_summary = TensorFlow.summary.scalar("x", x)
y_summary = TensorFlow.summary.scalar("y", y)

merged_summary_op = TensorFlow.summary.merge_all()

for (ii, x_val) in enumerate(-1:0.1:1)
    y_val, summaries = run(sess, [y, merged_summary_op], Dict(x=>x_val))
    write(summary_writer, summaries,  ii)
end

y = <Tensor y:1 shape=unknown dtype=Float64>


In [22]:
TensorFlow.load_python_process(;force_reload=true)

4

In [20]:
fetch(summary_writer.pyo)

┌ Error: Fatal error on process 3
│   exception =
│    PyError ($(Expr(:escape, :(ccall(#= /Users/oxinabox/.julia/packages/PyCall/0jMpb/src/pyfncall.jl:44 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <type 'exceptions.TypeError'>
│    TypeError("can't pickle SwigPyObject objects",)
│      File "/Users/oxinabox/.julia/conda/3/lib/python2.7/copy_reg.py", line 70, in _reduce_ex
│        raise TypeError, "can't pickle %s objects" % base.__name__
│    
│    Stacktrace:
│     [1] pyerr_check at /Users/oxinabox/.julia/packages/PyCall/0jMpb/src/exception.jl:60 [inlined]
│     [2] pyerr_check at /Users/oxinabox/.julia/packages/PyCall/0jMpb/src/exception.jl:64 [inlined]
│     [3] macro expansion at /Users/oxinabox/.julia/packages/PyCall/0jMpb/src/exception.jl:84 [inlined]
│     [4] __pycall!(::PyObject, ::Ptr{PyCall.PyObject_struct}, ::PyObject, ::Ptr{Nothing}) at /Users/oxinabox/.julia/packages/PyCall/0jMpb/src/pyfncall.jl:44
│     [5] _pycall!(::PyObject, ::Py

ProcessExitedException: ProcessExitedException()

## Automatic Node Naming

 - Notice before I did `@tf begin ... end`
 - **This is not at all required**
 - But it does enable automatic node naming
 - so `@tf y = sin(x)` actually becomes `y = sin(x; name="y")`
 - This gives you a good graph in tensorboard, and also better error messages.
 - Further it lets us look up tensors from the graph by **name**

In [6]:
@show sess.graph["x"]
@show sess.graph["y"]

run(sess, sess.graph["y"], Dict(sess.graph["x"]=>0.5))

sess.graph["x"] = <Tensor x:1 shape=unknown dtype=Float64>
sess.graph["y"] = <Tensor y:1 shape=unknown dtype=Float64>


0.479425538604203

In [ ]:
using TensorFlow
using MLDataUtils
using MLDatasets

using ProgressMeter

┌ Info: Recompiling stale cache file /Users/oxinabox/.julia/compiled/v1.1/MLDataUtils/CQWB9.ji for MLDataUtils [cc2ba9b6-d476-5e6d-8eaf-a92d5412d41d]
└ @ Base loading.jl:1184
┌ Warning: Module Compat with build ID 34372805313397 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Warning: Module Compat with build ID 34372805313397 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1186
┌ Warning: Module Compat with build ID 34372805313397 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
┌ Info: Recompi

In [8]:
leaky_relu6(x) = 0.01x + nn.relu6(x)

Internal error: encountered unexpected error in runtime:
InterruptException()
jl_mutex_unlock at /Users/osx/buildbot/slave/package_osx64/build/src/./locks.h:138 [inlined]
jl_typeinf_end at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:2610
typeinf_ext at ./compiler/typeinfer.jl:577
typeinf_ext at ./compiler/typeinfer.jl:613
jfptr_typeinf_ext_1.clone_1 at /Applications/Julia-1.1.app/Contents/Resources/julia/lib/julia/sys.dylib (unknown line)
jl_apply_generic at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:2219 [inlined]
jl_apply at /Users/osx/buildbot/slave/package_osx64/build/src/./julia.h:1571 [inlined]
jl_type_infer at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:277
jl_compile_method_internal at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:1819
jl_fptr_trampoline at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:1863
jl_apply at /Users/osx/buildbot/slave/package_osx64/build/src/./julia.h:1571 [inlined]
jl_f__apply at /Users/osx/buildbot

leaky_relu6 (generic function with 1 method)

In [9]:
sess = Session(Graph())

# Network Definition
@tf begin
    X = placeholder(Float32, shape=[-1, 28*28])
    
    # Network parameters
    
    hl_sizes = [512, 128, 64, 2, 64, 128, 512]

    Zs = [X]
    for (ii, hlsize) in enumerate(hl_sizes)
        Wii = get_variable("W_$ii", [get_shape(Zs[end], 2), hlsize], Float32)
        bii = get_variable("b_$ii", [hlsize], Float32)
        Zii = leaky_relu6(Zs[end]*Wii + bii)
        push!(Zs, Zii)
    end
    
    Wout = get_variable([get_shape(Zs[end], 2), 28*28], Float32)
    bout = get_variable([28*28], Float32)
    Y = nn.sigmoid(Zs[end]*Wout + bout)
    
    
    
    Z_code = Zs[end÷2] # A name for the coding layer
    has_died = reduce_any(reduce_all(Z_code.==0f0, axis=2))
end


Internal error: encountered unexpected error in runtime:
InterruptException()
obviously_disjoint at /Users/osx/buildbot/slave/package_osx64/build/src/subtype.c:277
jl_type_intersection_env_s at /Users/osx/buildbot/slave/package_osx64/build/src/subtype.c:2357
jl_typemap_intersection_node_visitor at /Users/osx/buildbot/slave/package_osx64/build/src/typemap.c:479
jl_typemap_intersection_visitor at /Users/osx/buildbot/slave/package_osx64/build/src/typemap.c:553
ml_matches at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:2562
jl_matching_methods at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:1765
abstract_call_gf_by_type at ./reflection.jl:790
abstract_call at ./compiler/abstractinterpretation.jl:776
abstract_eval_call at ./compiler/abstractinterpretation.jl:805
abstract_eval at ./compiler/abstractinterpretation.jl:890
typeinf_local at ./compiler/abstractinterpretation.jl:1135
typeinf_nocycle at ./compiler/abstractinterpretation.jl:1191
typeinf at ./compiler/typeinfer.jl

<Tensor has_died:1 shape=() dtype=Bool>

In [10]:
losses = 0.5(Y .- X).^2
loss = reduce_mean(losses)

<Tensor reduce_2:1 shape=() dtype=Float64>

In [11]:
#optimizer = train.minimize(train.AdamOptimizer(), loss)
optimizer = train.minimize(train.GradientDescentOptimizer(0.00001), loss)


ProcessExitedException: ProcessExitedException()

In [12]:

run(sess, global_variables_initializer())
auto_loss = Float64[]
@showprogress for epoch in 1:75
    epoch_loss = Float64[]
    for batch_x in eachbatch(train_images, 1_000, ObsDim.Last())
        loss_o, _ = run(sess, (loss, optimizer), Dict(X=>batch_x'))
        push!(epoch_loss, loss_o)
    end
    push!(auto_loss, mean(epoch_loss))
    #println("Epoch $epoch loss: $(auto_loss[end])")
    
    ### Check to see if it died
    if run(sess, has_died, Dict(X=>train_images'))
        error("Neuron in hidden layer has died, must reinitialize.")
    end
end

LoadError: UndefVarError: @showprogress not defined

In [13]:
pkg"add MLDataUtils"

Internal error: encountered unexpected error in runtime:
InterruptException()
has_free_typevars at /Users/osx/buildbot/slave/package_osx64/build/src/jltypes.c:151
subtype_tuple at /Users/osx/buildbot/slave/package_osx64/build/src/subtype.c:864 [inlined]
subtype at /Users/osx/buildbot/slave/package_osx64/build/src/subtype.c:1011
subtype_unionall at /Users/osx/buildbot/slave/package_osx64/build/src/subtype.c:650
exists_subtype at /Users/osx/buildbot/slave/package_osx64/build/src/subtype.c:1092 [inlined]
forall_exists_subtype at /Users/osx/buildbot/slave/package_osx64/build/src/subtype.c:1120
jl_subtype_env at /Users/osx/buildbot/slave/package_osx64/build/src/subtype.c:1180
jl_type_intersection_env_s at /Users/osx/buildbot/slave/package_osx64/build/src/subtype.c:2366
jl_typemap_intersection_node_visitor at /Users/osx/buildbot/slave/package_osx64/build/src/typemap.c:479
jl_typemap_intersection_visitor at /Users/osx/buildbot/slave/package_osx64/build/src/typemap.c:553
jl_typemap_intersectio

InterruptException: InterruptException:

Internal error: encountered unexpected error in runtime:
InterruptException()
jl_mutex_unlock at /Users/osx/buildbot/slave/package_osx64/build/src/./locks.h:138 [inlined]
jl_typeinf_end at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:2610
typeinf_ext at ./compiler/typeinfer.jl:577
typeinf_ext at ./compiler/typeinfer.jl:613
jfptr_typeinf_ext_1.clone_1 at /Applications/Julia-1.1.app/Contents/Resources/julia/lib/julia/sys.dylib (unknown line)
jl_apply_generic at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:2219 [inlined]
jl_apply at /Users/osx/buildbot/slave/package_osx64/build/src/./julia.h:1571 [inlined]
jl_type_infer at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:277
jl_compile_method_internal at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:1819
jl_fptr_trampoline at /Users/osx/buildbot/slave/package_osx64/build/src/gf.c:1863
show_backtrace at ./errorshow.jl:582
show_bt at /Users/oxinabox/.julia/packages/IJulia/fjEtl/src/display.jl:135
unknown f

# TensorFlow.jl Conventions vs Julia Conventions vs Python TensorFlow Conventions


| **Julia**                                                                                                           | **Python TensorFlow**                                                                                                                   |  **TensorFlow.jl**                                                                                                                                                                                                                 |
|---------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1-based indexing                                                                                                    |  0-based indexing                                                                                                                       | 1-based indexing                                                                                                                                                                                                                         |
| explicit broadcasting                                                                                               |  implicit broadcasting                                                                                                                  |  implicit or explicit broadcasting                                                                                                                                                                                                       |
| last index at `end`, 2nd last in `end-1`, etc.                                                                             |  last index at `-1` second last in `-2`                                                                                                 | last index at `end` 2nd last in `end-1`                                                                                                                                                                                                  |
| Operations in Julia ecosystem namespaces. (`SVD` in `LinearAlgebra`, `erfc` in `SpecialFunctions`, `cos` in `Base`) |  All operations in TensorFlow's namespaces  (`SVD` in `tf.linalg`, `erfc` in `tf.math`,  `cos` in `tf.math`, and all reexported from `tf`) |  Existing Julia functions overloaded to call TensorFlow equivalents when called with TensorFlow arguments   |                                                                                                                                                                                                                           |
| Container types are parametrized by number of dimensions and element type                                           |  N/A: does not have a parametric type system                                                                                            | Tensors are parametrized by element type, enabling easy specialization of algorithms for different types.                                                                                                                                                                                                 |


Python TensorFlow almost ended up a lot more like Julia 1.0,
but then they worried about scaring the NumPys off.

# Invenia Labs
## We're hiring
### People who know Julia
### People who know Machine Learning
I have left some fliers about open positions at the entrance.